In [1]:
import os
BUCKET = os.getenv('WORKSPACE_BUCKET')
!git clone https://github.com/EichlerLab/AoU_WDL.git
!gsutil rsync -r AoU_WDL/bam_to_contig {BUCKET}/cromwell_input/bam_to_contig

Cloning into 'AoU_WDL'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 324 (delta 29), reused 48 (delta 14), pack-reused 255
Receiving objects: 100% (324/324), 61.45 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (165/165), done.
Building synchronization state...
Starting synchronization...
Copying file://AoU_WDL/bam_to_contig/bam_to_contig.json [Content-Type=application/json]...
Copying file://AoU_WDL/bam_to_contig/bam_to_contig.sh [Content-Type=text/x-sh]...
Copying file://AoU_WDL/bam_to_contig/bam_to_contig.wdl [Content-Type=application/octet-stream]...
Copying file://AoU_WDL/bam_to_contig/concat_contigs.py [Content-Type=text/x-python]...
/ [4 files][  8.1 KiB/  8.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rsync ... Please
see the -m section under

In [7]:
!gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/pooled/longreads/v7_base/hifiasm/ | xargs -n 1 basename > samples.txt

In [8]:
# Extract haplotigs at locus of interest
import os

BUCKET = os.getenv('WORKSPACE_BUCKET')
with open('fmr1_t2t.bed', 'w') as out_f:
    out_f.write('chrX\t146176546\t146176888\n')
#FMR1        chrX:146176546-146176888 (UTR through Exon 2)
#HTT         chr4:3073552-3073843 (Exon 1-2)
GENE_COORDS_BED = 'fmr1_t2t.bed'
with open(GENE_COORDS_BED) as f:
    coords = f.readline().strip().split()
    FLANK_BP = int(coords[2]) - int(coords[1])

with open('samples.txt') as f:
     samples = f.readlines()
        
with open('ordered_sample_names_w_hap.txt', 'w') as out_sample, open('ordered_haplotig_fasta_names.txt', 'w') as out_fasta, open('ordered_bam_names.txt', 'w') as out_bam, open('ordered_bai_names.txt', 'w') as out_bai:
    for sample in samples:
        for hap in ['1', '2']:
            out_sample.write(f'{sample}_hap{hap}\n')
            out_fasta.write(f'gs://fc-aou-preprod-datasets-controlled/pooled/longreads/v7_base/assembly/hifiasm/{sample}/{sample}.haploTigs/{sample}.bp.hap{hap}.p_ctg.fa.gz\n')
            out_bam.write(f'{BUCKET}/alpha2-t2t/asm_align/bam/{sample}-asm_h{hap}.minimap2.bam\n')
            out_bai.write(f'{BUCKET}/alpha2-t2t/asm_align/bam/{sample}-asm_h{hap}.minimap2.bam.bai\n')

In [9]:
!gsutil cp {GENE_COORDS_BED} {BUCKET}/cromwell_input/bam_to_contig/{GENE_COORDS_BED}

Copying file://fmr1_t2t.bed [Content-Type=application/octet-stream]...
/ [1 files][   25.0 B/   25.0 B]                                                
Operation completed over 1 objects/25.0 B.                                       


In [10]:
import json
d = {
  "bam_to_contig.bam_to_contig_bash_script": f"{BUCKET}/cromwell_input/bam_to_contig/bam_to_contig.sh",
  "bam_to_contig.make_contig_bed_script": f"{BUCKET}/cromwell_input/bam_to_contig/make_contig_bed.py",
  "bam_to_contig.ordered_sample_names_w_hap": f"{BUCKET}/cromwell_input/bam_to_contig/ordered_sample_names_w_hap.txt",
  "bam_to_contig.ordered_bams": f"{BUCKET}/cromwell_input/bam_to_contig/ordered_bam_names.txt",
  "bam_to_contig.ordered_bais": f"{BUCKET}/cromwell_input/bam_to_contig/ordered_bai_names.txt",
  "bam_to_contig.ordered_haplotig_fastas": f"{BUCKET}/cromwell_input/bam_to_contig/ordered_haplotig_fasta_names.txt",
  "bam_to_contig.regions_bed": f"{BUCKET}/cromwell_input/bam_to_contig/{GENE_COORDS_BED}",
  "bam_to_contig.flank_bp": f"{FLANK_BP}",
  "bam_to_contig.run_faidx_script": f"{BUCKET}/cromwell_input/bam_to_contig/run_samtools_faidx_for_sample.sh",
  "bam_to_contig.concat_contigs_script": f"{BUCKET}/cromwell_input/bam_to_contig/concat_contigs.py"
}
with open('bam_to_contig.json', 'w') as out_f:
    out_f.write(json.dumps(d))

In [12]:
import os
import subprocess
import json
import requests
from pathlib import Path

def check_for_app(env):
    list_apps_url = f'{env["leonardo_url"]}/api/google/v1/apps/{env["google_project"]}'
    r = requests.get(
        list_apps_url,
        params={
          'includeDeleted': 'false'
        },
        headers = {
            'Authorization': f'Bearer {env["token"]}'
        }
    )
    r.raise_for_status()

    for potential_app in r.json():
        if potential_app['appType'] == 'CROMWELL' and (
                str(potential_app['auditInfo']['creator']) == env['owner_email']
                or str(potential_app['auditInfo']['creator']) == env['user_email']
        ) :
            potential_app_name = potential_app['appName']
            potential_app_status = potential_app['status']

            # We found a CROMWELL app in the correct google project and owned by the user. Now just check the workspace:
            _, workspace_namespace,  proxy_url = get_app_details(env, potential_app_name)
            if workspace_namespace == env['workspace_namespace']:
                return potential_app_name, potential_app_status, proxy_url['cromwell-service']

    return None, None, None

def get_app_details(env, app_name):
    get_app_url = f'{env["leonardo_url"]}/api/google/v1/apps/{env["google_project"]}/{app_name}'
    print('start')
    r = requests.get(
        get_app_url,
        params={
            'includeDeleted': 'true',
            'role': 'creator'
        },
        headers={
            'Authorization': f'Bearer {env["token"]}'
        }
    )
    if r.status_code == 404:
        return 'DELETED', None, None, None
    else:
        r.raise_for_status()
    result_json = r.json()
    custom_environment_variables = result_json['customEnvironmentVariables']
    return result_json['status'], custom_environment_variables['WORKSPACE_NAMESPACE'], result_json.get('proxyUrls')

# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell():
    if validate_cromshell_alias():
        print("Found cromshell, please use cromshell")
    elif validate_cromshell_alpha():
        print("Found cromshell-alpha, please use cromshell-alpha")
    else:
        raise Exception("Cromshell is not installed.")

# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell_alpha():
    print('Scanning for cromshell 2 alpha...')
    try:
        subprocess.run(['cromshell-alpha', 'version'], capture_output=True, check=True, encoding='utf-8')
    except FileNotFoundError:
        return False
    return True
# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell_alias():
    print('Scanning for cromshell 2')
    try:
        subprocess.run(['cromshell', 'version'], capture_output=True, check=True, encoding='utf-8')
    except FileNotFoundError:
        return False
    return True

def configure_cromwell(env, proxy_url):
     print('Updating cromwell config')
     file = f'{str(Path.home())}/.cromshell/cromshell_config.json'
     configuration = {
        'cromwell_server': proxy_url.split("swagger/", 1)[0] if proxy_url else "invalid url",
        'requests_timeout': 5,
        'gcloud_token_email': env['user_email'],
        'referer_header_url': env['leonardo_url']
     }
     with open(file, 'w') as filetowrite:
        filetowrite.write(json.dumps(configuration, indent=2))

def find_app_status(env):
    print(f'Checking status for CROMWELL app')
    app_name, app_status, proxy_url = check_for_app(env)

    configure_cromwell(env, proxy_url)

    if app_name is None:
        print(f'CROMWELL app does not exist. Please create cromwell server from workbench')
    else:
        print(f'app_name={app_name}; app_status={app_status}')
        print(f'Existing CROMWELL app found (app_name={app_name}; app_status={app_status}).')
        exit(1)

def main():
    # Iteration 1: these ENV reads will throw errors if not set.
    env = {
        'workspace_namespace': os.environ['WORKSPACE_NAMESPACE'],
        'workspace_bucket': os.environ['WORKSPACE_BUCKET'],
        'user_email': os.environ.get('PET_SA_EMAIL', default = os.environ['OWNER_EMAIL']),
        'owner_email': os.environ['OWNER_EMAIL'],
        'google_project': os.environ['GOOGLE_PROJECT'],
        'leonardo_url': os.environ['LEONARDO_BASE_URL']
    }

    # Before going any further, check that cromshell2 is installed:
    validate_cromshell()

    # Fetch the token:
    token_fetch_command = subprocess.run(['gcloud', 'auth', 'print-access-token', env['user_email']], capture_output=True, check=True, encoding='utf-8')
    env['token'] = str.strip(token_fetch_command.stdout)

    find_app_status(env)


if __name__ == '__main__':
    main()
 


Scanning for cromshell 2
Found cromshell, please use cromshell
Checking status for CROMWELL app
Updating cromwell config
CROMWELL app does not exist. Please create cromwell server from workbench


In [ ]:
!cromshell submit AoU_WDL/bam_to_contig/bam_to_contig.wdl bam_to_contig.json

In [ ]:
!git clone https://github.com/EichlerLab/compteam_tools.git
!python compteam_tools/visualize_repeat_region/make_viz.py compteam_tools/visualize_repeat_region/fmr1_params_example.json